# Sharding Patterns: Range, Hash, and Directory

**Sharding** (also known as horizontal partitioning) is a database scaling strategy that distributes data across multiple servers or nodes. Each shard holds a subset of the total data, enabling:

- **Horizontal scalability** – Add more nodes to handle increased load
- **Improved performance** – Queries target specific shards
- **Fault isolation** – Failure in one shard doesn't affect others

This notebook explores three fundamental sharding strategies:

| Strategy | Description | Best For |
|----------|-------------|----------|
| **Range Sharding** | Data partitioned by value ranges | Time-series, sequential data |
| **Hash Sharding** | Data distributed using hash function | Even distribution, random access |
| **Directory Sharding** | Lookup table maps keys to shards | Complex routing rules, multi-tenant |

---
## 1. Range Sharding

Range sharding partitions data based on ranges of a **shard key** value. Each shard is responsible for a contiguous range.

### Characteristics
- ✅ **Range queries are efficient** – Data for a range resides on one or few shards
- ✅ **Easy to understand** – Clear boundaries between shards
- ❌ **Hot spots** – Uneven data distribution if ranges are poorly chosen
- ❌ **Sequential inserts** – New data may overwhelm a single shard

### Example Use Cases
- Time-series data (partition by date/month)
- Geographic data (partition by region)
- Alphabetical data (partition by first letter)

In [ ]:
from typing import Any, Dict, List, Tuple, Optional
from dataclasses import dataclass, field
import hashlib
import bisect


@dataclass
class RangeShard:
    """A shard that handles a specific range of keys."""
    name: str
    min_key: int  # Inclusive
    max_key: int  # Exclusive
    data: Dict[int, Any] = field(default_factory=dict)
    
    def contains(self, key: int) -> bool:
        return self.min_key <= key < self.max_key
    
    def insert(self, key: int, value: Any) -> None:
        if not self.contains(key):
            raise ValueError(f"Key {key} outside range [{self.min_key}, {self.max_key})")
        self.data[key] = value
    
    def get(self, key: int) -> Optional[Any]:
        return self.data.get(key)
    
    def __len__(self) -> int:
        return len(self.data)


class RangeShardedDB:
    """Database with range-based sharding."""
    
    def __init__(self, shard_ranges: List[Tuple[str, int, int]]):
        """
        Initialize with shard definitions.
        
        Args:
            shard_ranges: List of (shard_name, min_key, max_key) tuples
        """
        self.shards = [
            RangeShard(name=name, min_key=min_k, max_key=max_k)
            for name, min_k, max_k in shard_ranges
        ]
        # Sort by min_key for efficient lookup
        self.shards.sort(key=lambda s: s.min_key)
        self._boundaries = [s.min_key for s in self.shards]
    
    def _get_shard(self, key: int) -> RangeShard:
        """Find the shard responsible for a given key."""
        idx = bisect.bisect_right(self._boundaries, key) - 1
        if idx < 0 or idx >= len(self.shards):
            raise ValueError(f"No shard found for key {key}")
        shard = self.shards[idx]
        if not shard.contains(key):
            raise ValueError(f"No shard found for key {key}")
        return shard
    
    def insert(self, key: int, value: Any) -> str:
        """Insert a key-value pair, returns shard name."""
        shard = self._get_shard(key)
        shard.insert(key, value)
        return shard.name
    
    def get(self, key: int) -> Tuple[Optional[Any], str]:
        """Get a value by key, returns (value, shard_name)."""
        shard = self._get_shard(key)
        return shard.get(key), shard.name
    
    def range_query(self, start: int, end: int) -> List[Tuple[int, Any, str]]:
        """Query a range of keys, returns list of (key, value, shard_name)."""
        results = []
        for shard in self.shards:
            # Check if shard range overlaps with query range
            if shard.max_key <= start or shard.min_key >= end:
                continue
            for key, value in shard.data.items():
                if start <= key < end:
                    results.append((key, value, shard.name))
        return sorted(results, key=lambda x: x[0])
    
    def get_distribution(self) -> Dict[str, int]:
        """Get the number of records in each shard."""
        return {shard.name: len(shard) for shard in self.shards}

In [ ]:
# Example: Range sharding by user ID
range_db = RangeShardedDB([
    ("shard_A", 0, 1000),      # Users 0-999
    ("shard_B", 1000, 2000),   # Users 1000-1999
    ("shard_C", 2000, 3000),   # Users 2000-2999
])

# Insert sample data
import random
random.seed(42)

users = [
    (random.randint(0, 2999), f"user_{i}@example.com")
    for i in range(50)
]

for user_id, email in users:
    shard = range_db.insert(user_id, {"email": email, "id": user_id})
    
# Check distribution
print("📊 Data Distribution:")
for shard_name, count in range_db.get_distribution().items():
    print(f"  {shard_name}: {count} records")

# Perform a range query
print("\n🔍 Range Query (keys 500-1500):")
results = range_db.range_query(500, 1500)
for key, value, shard in results[:5]:  # Show first 5
    print(f"  Key {key} -> {shard}")

---
## 2. Hash Sharding

Hash sharding uses a **hash function** to determine which shard holds a piece of data. The hash of the shard key is computed, then mapped to a shard (typically using modulo).

### Characteristics
- ✅ **Even distribution** – Hash functions spread data uniformly
- ✅ **No hot spots** – Sequential keys are distributed across shards
- ❌ **Range queries inefficient** – Must query all shards
- ❌ **Resharding is expensive** – Adding shards requires data migration

### Hash Function Formula
$$\text{shard\_index} = \text{hash}(\text{key}) \mod N$$

Where $N$ is the number of shards.

In [ ]:
@dataclass
class HashShard:
    """A shard in a hash-partitioned database."""
    name: str
    index: int
    data: Dict[str, Any] = field(default_factory=dict)
    
    def insert(self, key: str, value: Any) -> None:
        self.data[key] = value
    
    def get(self, key: str) -> Optional[Any]:
        return self.data.get(key)
    
    def __len__(self) -> int:
        return len(self.data)


class HashShardedDB:
    """Database with hash-based sharding."""
    
    def __init__(self, num_shards: int):
        self.num_shards = num_shards
        self.shards = [
            HashShard(name=f"shard_{i}", index=i)
            for i in range(num_shards)
        ]
    
    def _hash_key(self, key: str) -> int:
        """Compute hash of a key."""
        # Using MD5 for consistent hashing (not for security)
        hash_bytes = hashlib.md5(str(key).encode()).digest()
        return int.from_bytes(hash_bytes[:4], 'big')
    
    def _get_shard_index(self, key: str) -> int:
        """Determine which shard holds a key."""
        return self._hash_key(key) % self.num_shards
    
    def _get_shard(self, key: str) -> HashShard:
        """Get the shard for a key."""
        return self.shards[self._get_shard_index(key)]
    
    def insert(self, key: str, value: Any) -> str:
        """Insert a key-value pair."""
        shard = self._get_shard(key)
        shard.insert(key, value)
        return shard.name
    
    def get(self, key: str) -> Tuple[Optional[Any], str]:
        """Get a value by key."""
        shard = self._get_shard(key)
        return shard.get(key), shard.name
    
    def get_distribution(self) -> Dict[str, int]:
        """Get the number of records in each shard."""
        return {shard.name: len(shard) for shard in self.shards}

In [ ]:
# Example: Hash sharding with 4 shards
hash_db = HashShardedDB(num_shards=4)

# Insert sequential user IDs (would cause hot spots in range sharding)
for i in range(100):
    user_id = f"user_{i:05d}"
    hash_db.insert(user_id, {"name": f"User {i}", "id": user_id})

# Check distribution - should be relatively even
print("📊 Hash Sharding Distribution (100 sequential keys):")
distribution = hash_db.get_distribution()
for shard_name, count in distribution.items():
    bar = "█" * count
    print(f"  {shard_name}: {count:3d} {bar}")

# Demonstrate consistent routing
print("\n🔗 Key Routing Examples:")
for key in ["user_00042", "user_00099", "user_00001"]:
    value, shard = hash_db.get(key)
    print(f"  {key} -> {shard}")

### ⚠️ The Resharding Problem

When adding or removing shards with simple modulo hashing, **most keys get remapped**:

```
Before (3 shards): hash(key) % 3 = 2
After  (4 shards): hash(key) % 4 = 1  ← Different shard!
```

This requires massive data migration. **Consistent hashing** solves this problem.

In [ ]:
# Demonstrate the resharding problem
def simulate_resharding(keys: List[str], old_shards: int, new_shards: int) -> float:
    """Calculate percentage of keys that change shards."""
    def hash_key(key: str) -> int:
        return int.from_bytes(hashlib.md5(key.encode()).digest()[:4], 'big')
    
    moved = 0
    for key in keys:
        h = hash_key(key)
        old_shard = h % old_shards
        new_shard = h % new_shards
        if old_shard != new_shard:
            moved += 1
    
    return (moved / len(keys)) * 100

# Test with 10,000 keys
test_keys = [f"key_{i}" for i in range(10000)]

print("📦 Data Migration with Simple Modulo Hashing:")
print("─" * 45)
for old, new in [(3, 4), (4, 5), (5, 6), (10, 11)]:
    pct = simulate_resharding(test_keys, old, new)
    print(f"  {old} → {new} shards: {pct:.1f}% keys need to move")

---
## 3. Consistent Hashing

Consistent hashing minimizes key redistribution when nodes are added or removed. Instead of modulo, it uses a **hash ring**.

### How It Works
1. Both **nodes** and **keys** are hashed to positions on a virtual ring (0 to 2³² - 1)
2. A key is assigned to the **first node clockwise** from its position
3. When a node is added/removed, only keys between it and its neighbor move

### Virtual Nodes (Vnodes)
To improve distribution, each physical node is represented by multiple **virtual nodes** on the ring.

In [ ]:
class ConsistentHashRing:
    """Consistent hashing implementation with virtual nodes."""
    
    def __init__(self, num_virtual_nodes: int = 150):
        self.num_virtual_nodes = num_virtual_nodes
        self.ring: Dict[int, str] = {}  # hash -> node_name
        self.sorted_hashes: List[int] = []
        self.nodes: set = set()
    
    def _hash(self, key: str) -> int:
        """Hash a key to a position on the ring."""
        return int(hashlib.md5(key.encode()).hexdigest(), 16)
    
    def add_node(self, node: str) -> None:
        """Add a node with virtual nodes to the ring."""
        self.nodes.add(node)
        for i in range(self.num_virtual_nodes):
            virtual_key = f"{node}:vnode{i}"
            h = self._hash(virtual_key)
            self.ring[h] = node
            bisect.insort(self.sorted_hashes, h)
    
    def remove_node(self, node: str) -> None:
        """Remove a node from the ring."""
        self.nodes.discard(node)
        for i in range(self.num_virtual_nodes):
            virtual_key = f"{node}:vnode{i}"
            h = self._hash(virtual_key)
            if h in self.ring:
                del self.ring[h]
                self.sorted_hashes.remove(h)
    
    def get_node(self, key: str) -> Optional[str]:
        """Get the node responsible for a key."""
        if not self.ring:
            return None
        
        h = self._hash(key)
        # Find the first node clockwise from the key's position
        idx = bisect.bisect(self.sorted_hashes, h)
        if idx >= len(self.sorted_hashes):
            idx = 0  # Wrap around
        
        return self.ring[self.sorted_hashes[idx]]
    
    def get_key_distribution(self, keys: List[str]) -> Dict[str, int]:
        """Get the distribution of keys across nodes."""
        distribution = {node: 0 for node in self.nodes}
        for key in keys:
            node = self.get_node(key)
            if node:
                distribution[node] += 1
        return distribution

In [ ]:
# Example: Consistent hashing in action
ring = ConsistentHashRing(num_virtual_nodes=150)

# Add initial nodes
for node in ["node_A", "node_B", "node_C"]:
    ring.add_node(node)

# Generate test keys
test_keys = [f"user:{i}" for i in range(10000)]

# Check initial distribution
print("📊 Initial Distribution (3 nodes):")
initial_dist = ring.get_key_distribution(test_keys)
for node, count in sorted(initial_dist.items()):
    pct = count / len(test_keys) * 100
    bar = "█" * int(pct)
    print(f"  {node}: {count:5d} ({pct:.1f}%) {bar}")

# Record which node each key maps to
initial_mapping = {key: ring.get_node(key) for key in test_keys}

In [ ]:
# Add a new node and measure migration
ring.add_node("node_D")

# Check new distribution
print("📊 Distribution After Adding node_D (4 nodes):")
new_dist = ring.get_key_distribution(test_keys)
for node, count in sorted(new_dist.items()):
    pct = count / len(test_keys) * 100
    bar = "█" * int(pct)
    print(f"  {node}: {count:5d} ({pct:.1f}%) {bar}")

# Calculate migration
moved_keys = sum(
    1 for key in test_keys 
    if ring.get_node(key) != initial_mapping[key]
)
migration_pct = moved_keys / len(test_keys) * 100

print(f"\n📦 Keys Migrated: {moved_keys:,} ({migration_pct:.1f}%)")
print(f"   Optimal (1/N): {100/4:.1f}%")
print(f"   vs Simple Hash: ~75%")

### Visualizing the Hash Ring

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_hash_ring(ring: ConsistentHashRing, sample_keys: List[str] = None):
    """Visualize the consistent hash ring."""
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Draw the ring
    theta = np.linspace(0, 2 * np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'k-', linewidth=2, alpha=0.3)
    
    # Color map for nodes
    colors = plt.cm.Set2(np.linspace(0, 1, len(ring.nodes)))
    node_colors = {node: colors[i] for i, node in enumerate(sorted(ring.nodes))}
    
    # Plot virtual nodes (sample only to avoid clutter)
    max_ring = 2**128
    sample_vnodes = list(ring.ring.items())[::10]  # Every 10th vnode
    
    for h, node in sample_vnodes:
        angle = (h / max_ring) * 2 * np.pi
        x, y = np.cos(angle) * 1.05, np.sin(angle) * 1.05
        ax.scatter(x, y, c=[node_colors[node]], s=30, alpha=0.6, zorder=5)
    
    # Plot sample keys if provided
    if sample_keys:
        for key in sample_keys[:20]:  # Limit to 20 keys
            h = ring._hash(key)
            angle = (h / max_ring) * 2 * np.pi
            x, y = np.cos(angle) * 0.85, np.sin(angle) * 0.85
            
            node = ring.get_node(key)
            ax.scatter(x, y, c=[node_colors[node]], s=100, marker='*', 
                      edgecolors='black', linewidth=0.5, zorder=10)
    
    # Create legend
    for node, color in node_colors.items():
        ax.scatter([], [], c=[color], s=100, label=node)
    ax.legend(loc='upper right', title='Nodes')
    
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Consistent Hash Ring\n(dots = vnodes, stars = sample keys)', fontsize=14)
    
    plt.tight_layout()
    plt.show()

# Visualize the ring
sample_keys = [f"key_{i}" for i in range(20)]
visualize_hash_ring(ring, sample_keys)

---
## 4. Directory-Based Sharding

Directory sharding uses a **lookup table** (directory) to map keys to shards. This provides maximum flexibility but adds a single point of lookup.

### Characteristics
- ✅ **Maximum flexibility** – Any routing logic can be implemented
- ✅ **Easy rebalancing** – Just update the directory
- ❌ **Lookup overhead** – Every operation requires directory access
- ❌ **Single point of failure** – Directory must be highly available

### Use Cases
- Multi-tenant applications (tenant → shard mapping)
- Complex routing rules
- Hybrid sharding strategies

In [ ]:
@dataclass
class DirectoryShard:
    """A shard in a directory-based sharded database."""
    name: str
    capacity: int
    data: Dict[str, Any] = field(default_factory=dict)
    
    @property
    def usage(self) -> float:
        return len(self.data) / self.capacity
    
    def is_full(self) -> bool:
        return len(self.data) >= self.capacity


class DirectoryShardedDB:
    """Database with directory-based sharding."""
    
    def __init__(self):
        self.shards: Dict[str, DirectoryShard] = {}
        self.directory: Dict[str, str] = {}  # key -> shard_name mapping
    
    def add_shard(self, name: str, capacity: int = 1000) -> None:
        """Add a new shard."""
        self.shards[name] = DirectoryShard(name=name, capacity=capacity)
    
    def _find_available_shard(self) -> Optional[DirectoryShard]:
        """Find a shard with capacity."""
        for shard in self.shards.values():
            if not shard.is_full():
                return shard
        return None
    
    def insert(self, key: str, value: Any) -> str:
        """Insert a key-value pair."""
        # Check if key already exists
        if key in self.directory:
            shard_name = self.directory[key]
            self.shards[shard_name].data[key] = value
            return shard_name
        
        # Find an available shard
        shard = self._find_available_shard()
        if not shard:
            raise RuntimeError("All shards are full!")
        
        # Insert and update directory
        shard.data[key] = value
        self.directory[key] = shard.name
        return shard.name
    
    def get(self, key: str) -> Tuple[Optional[Any], Optional[str]]:
        """Get a value by key."""
        shard_name = self.directory.get(key)
        if not shard_name:
            return None, None
        return self.shards[shard_name].data.get(key), shard_name
    
    def move_key(self, key: str, target_shard: str) -> bool:
        """Move a key to a different shard (rebalancing)."""
        if key not in self.directory:
            return False
        if target_shard not in self.shards:
            return False
        if self.shards[target_shard].is_full():
            return False
        
        # Move the data
        source_shard = self.directory[key]
        value = self.shards[source_shard].data.pop(key)
        self.shards[target_shard].data[key] = value
        self.directory[key] = target_shard
        return True
    
    def get_shard_stats(self) -> Dict[str, Dict]:
        """Get statistics for all shards."""
        return {
            name: {"count": len(shard.data), "usage": f"{shard.usage:.1%}"}
            for name, shard in self.shards.items()
        }

In [ ]:
# Example: Multi-tenant SaaS application
dir_db = DirectoryShardedDB()

# Add shards with different capacities
dir_db.add_shard("shard_premium", capacity=100)   # Premium tier
dir_db.add_shard("shard_standard_1", capacity=500)
dir_db.add_shard("shard_standard_2", capacity=500)

# Simulate tenant data insertion
tenants = {
    "acme_corp": "Enterprise",
    "startup_xyz": "Startup",
    "bigco_inc": "Enterprise",
    "small_biz": "SMB",
}

for tenant, tier in tenants.items():
    for i in range(50):
        key = f"{tenant}:data_{i}"
        shard = dir_db.insert(key, {"tenant": tenant, "tier": tier, "id": i})

print("📊 Shard Statistics:")
for shard, stats in dir_db.get_shard_stats().items():
    print(f"  {shard}: {stats['count']} records ({stats['usage']})")

# Demonstrate directory lookup
print("\n🔍 Directory Lookups:")
for key in ["acme_corp:data_0", "startup_xyz:data_25"]:
    value, shard = dir_db.get(key)
    print(f"  {key} -> {shard}")

---
## 5. Hot Spots and Rebalancing Challenges

### Hot Spots

A **hot spot** occurs when one shard receives disproportionately more traffic than others, causing:
- Performance degradation
- Increased latency
- Potential resource exhaustion

### Common Causes

| Cause | Example | Mitigation |
|-------|---------|------------|
| **Poor shard key** | Using timestamp for range sharding | Use hash sharding or composite keys |
| **Celebrity problem** | One user has millions of followers | Split hot keys across multiple shards |
| **Time-based access** | End-of-month reports | Pre-compute aggregates |
| **Geographic clustering** | All US users on one shard | Use consistent hashing |

In [ ]:
def simulate_hot_spot_scenario():
    """Simulate a hot spot caused by celebrity users."""
    
    # User follower counts (power law distribution)
    users = {
        "celebrity_A": 10_000_000,
        "celebrity_B": 5_000_000,
        "influencer_1": 500_000,
        "influencer_2": 250_000,
    }
    # Add 1000 regular users
    for i in range(1000):
        users[f"user_{i}"] = random.randint(10, 1000)
    
    # Hash shard by user ID
    num_shards = 4
    shard_load = {f"shard_{i}": 0 for i in range(num_shards)}
    
    def get_shard(user_id: str) -> str:
        h = int(hashlib.md5(user_id.encode()).hexdigest(), 16)
        return f"shard_{h % num_shards}"
    
    # Simulate read load (proportional to followers)
    for user_id, followers in users.items():
        shard = get_shard(user_id)
        shard_load[shard] += followers
    
    return shard_load, users

shard_load, users = simulate_hot_spot_scenario()
total_load = sum(shard_load.values())

print("🔥 Hot Spot Simulation (Social Media Reads):")
print("─" * 50)
for shard, load in sorted(shard_load.items()):
    pct = load / total_load * 100
    bar = "█" * int(pct / 2)
    status = "🔥 HOT!" if pct > 40 else ""
    print(f"  {shard}: {load:>12,} requests ({pct:5.1f}%) {bar} {status}")

# Show which celebrities are on which shard
print("\n👤 Celebrity Distribution:")
for user_id in ["celebrity_A", "celebrity_B", "influencer_1", "influencer_2"]:
    h = int(hashlib.md5(user_id.encode()).hexdigest(), 16)
    shard = f"shard_{h % 4}"
    print(f"  {user_id} ({users[user_id]:,} followers) -> {shard}")

### Rebalancing Strategies

**Rebalancing** is the process of redistributing data across shards to maintain even load.

In [ ]:
class RebalancingSimulator:
    """Simulate different rebalancing strategies."""
    
    def __init__(self, initial_shards: int = 4, keys_per_shard: int = 1000):
        self.shards = {f"shard_{i}": keys_per_shard for i in range(initial_shards)}
        self.total_keys = initial_shards * keys_per_shard
    
    def add_shard_naive(self, new_shard: str) -> Dict[str, int]:
        """
        Naive approach: Rehash everything.
        Returns migration count per source shard.
        """
        # With simple modulo, ~75% of keys need to move
        old_count = len(self.shards)
        new_count = old_count + 1
        
        keys_to_move = int(self.total_keys * (1 - old_count / new_count))
        per_shard = keys_to_move // old_count
        
        migration = {shard: per_shard for shard in self.shards}
        self.shards[new_shard] = keys_to_move
        
        return migration
    
    def add_shard_consistent(self, new_shard: str) -> Dict[str, int]:
        """
        Consistent hashing: Only 1/N keys move.
        """
        new_count = len(self.shards) + 1
        keys_to_move = self.total_keys // new_count
        
        # Keys come evenly from existing shards
        per_shard = keys_to_move // len(self.shards)
        
        migration = {shard: per_shard for shard in self.shards}
        self.shards[new_shard] = keys_to_move
        
        return migration

# Compare approaches
print("📦 Rebalancing Comparison (4 → 5 shards, 4000 keys):")
print("=" * 55)

sim_naive = RebalancingSimulator(4, 1000)
naive_migration = sim_naive.add_shard_naive("shard_4")

sim_consistent = RebalancingSimulator(4, 1000)
consistent_migration = sim_consistent.add_shard_consistent("shard_4")

print("\n🔄 Naive (Modulo Rehash):")
total_naive = sum(naive_migration.values())
for shard, count in naive_migration.items():
    print(f"  {shard} → shard_4: {count} keys")
print(f"  Total migrated: {total_naive} ({total_naive/4000*100:.1f}%)")

print("\n✅ Consistent Hashing:")
total_consistent = sum(consistent_migration.values())
for shard, count in consistent_migration.items():
    print(f"  {shard} → shard_4: {count} keys")
print(f"  Total migrated: {total_consistent} ({total_consistent/4000*100:.1f}%)")

print(f"\n📊 Migration Reduction: {(1 - total_consistent/total_naive)*100:.0f}%")

---
## 6. Choosing the Right Sharding Strategy

| Factor | Range | Hash | Directory | Consistent Hash |
|--------|-------|------|-----------|----------------|
| Range queries | ✅ Excellent | ❌ Poor | 🔶 Depends | ❌ Poor |
| Even distribution | ❌ Risk of hot spots | ✅ Good | ✅ Controllable | ✅ Good |
| Add/remove nodes | 🔶 Manual | ❌ Expensive | ✅ Easy | ✅ Minimal migration |
| Complexity | 🟢 Low | 🟢 Low | 🟡 Medium | 🟡 Medium |
| Lookup overhead | 🟢 O(log n) | 🟢 O(1) | 🟡 O(1) + network | 🟢 O(log n) |

In [ ]:
# Decision helper
def recommend_sharding_strategy(
    needs_range_queries: bool,
    high_write_volume: bool,
    frequent_scaling: bool,
    complex_routing: bool
) -> str:
    """Recommend a sharding strategy based on requirements."""
    
    recommendations = []
    
    if needs_range_queries and not high_write_volume:
        recommendations.append(("Range Sharding", 
            "Efficient range queries, but watch for hot spots"))
    
    if high_write_volume and not needs_range_queries:
        recommendations.append(("Hash Sharding", 
            "Even write distribution, but range queries are expensive"))
    
    if frequent_scaling:
        recommendations.append(("Consistent Hashing", 
            "Minimal data migration when scaling"))
    
    if complex_routing:
        recommendations.append(("Directory Sharding", 
            "Maximum flexibility for routing logic"))
    
    if not recommendations:
        recommendations.append(("Hash Sharding", 
            "Good default choice for most scenarios"))
    
    return recommendations

# Example scenarios
scenarios = [
    ("Time-series analytics", True, False, False, False),
    ("High-traffic social media", False, True, True, False),
    ("Multi-tenant SaaS", False, False, False, True),
    ("E-commerce catalog", False, True, True, False),
]

print("🧭 Sharding Strategy Recommendations:")
print("=" * 60)
for scenario, *args in scenarios:
    recs = recommend_sharding_strategy(*args)
    print(f"\n📋 {scenario}:")
    for strategy, reason in recs:
        print(f"   ✓ {strategy}: {reason}")

---
## 📝 Key Takeaways

### Sharding Strategies Summary

1. **Range Sharding**
   - Partitions by value ranges (e.g., date, alphabetical)
   - ✅ Great for range queries and sequential data
   - ⚠️ Prone to hot spots with uneven data distribution

2. **Hash Sharding**
   - Uses `hash(key) % N` for even distribution
   - ✅ Eliminates hot spots from sequential inserts
   - ⚠️ Range queries require scatter-gather across all shards
   - ⚠️ Adding shards causes massive data migration

3. **Consistent Hashing**
   - Uses a hash ring with virtual nodes
   - ✅ Only ~K/N keys move when adding a node
   - ✅ Industry standard for distributed caches (Redis, Memcached)
   - ⚠️ More complex to implement

4. **Directory Sharding**
   - Lookup table maps keys to shards
   - ✅ Maximum flexibility and easy rebalancing
   - ⚠️ Directory is a single point of failure
   - ⚠️ Every operation requires directory lookup

### Best Practices

- **Choose shard key carefully** – It determines data distribution and query patterns
- **Plan for growth** – Use consistent hashing if you expect to add nodes
- **Monitor hot spots** – Use metrics to detect uneven load
- **Consider composite keys** – Combine multiple fields for better distribution
- **Test rebalancing** – Ensure your system can handle shard additions gracefully

In [ ]:
# Quick reference cheat sheet
cheat_sheet = """
╔═══════════════════════════════════════════════════════════════════╗
║                    SHARDING PATTERNS CHEAT SHEET                  ║
╠═══════════════════════════════════════════════════════════════════╣
║  RANGE SHARDING                                                   ║
║  ─────────────                                                    ║
║  Formula: shard = find_range(key)                                 ║
║  Use for: Time-series, logs, sequential IDs                       ║
║  Avoid:   High-frequency inserts, celebrity data                  ║
╠═══════════════════════════════════════════════════════════════════╣
║  HASH SHARDING                                                    ║
║  ─────────────                                                    ║
║  Formula: shard = hash(key) % N                                   ║
║  Use for: Even distribution, random access                        ║
║  Avoid:   Range queries, frequent scaling                         ║
╠═══════════════════════════════════════════════════════════════════╣
║  CONSISTENT HASHING                                               ║
║  ──────────────────                                               ║
║  Formula: shard = first_node_clockwise(hash(key))                 ║
║  Use for: Caches, dynamic scaling, distributed systems            ║
║  Avoid:   Simple applications, when range queries needed          ║
╠═══════════════════════════════════════════════════════════════════╣
║  DIRECTORY SHARDING                                               ║
║  ──────────────────                                               ║
║  Formula: shard = directory[key]                                  ║
║  Use for: Multi-tenant, complex routing, hybrid approaches        ║
║  Avoid:   High-throughput with latency requirements               ║
╚═══════════════════════════════════════════════════════════════════╝
"""
print(cheat_sheet)